In [1]:
import pandas as pd

# Ler o CSV
df = pd.read_csv('comentarios.csv')

# Visualizar os dados
df = df[['product_name', 'review_text' ,'product_brand', 'site_category_lv1', 'site_category_lv2']]# Selecionar colunas

df['review_text'] = df['review_text'].astype(str)  # Garantir que os comentários sejam strings

# Salvar os dados limpos
df.to_csv('dados_processados.csv', index=False)

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Carregar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Carregar os dados processados
df = pd.read_csv('dados_processados.csv')

# Verificar e remover valores nulos
df = df.dropna(subset=['review_text'])

# Garantir que todos os comentários sejam strings
df['review_text'] = df['review_text'].astype(str)

# Transformar comentários em uma lista
comentarios = df['review_text'].tolist()

# Transformar comentários em vetores usando embeddings
embeddings = model.encode(comentarios)

# Configurar o índice FAISS
dimension = embeddings.shape[1]  # Dimensão dos vetores
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings, dtype=np.float32))

# Salvar o índice FAISS
faiss.write_index(index, 'faiss_index.index')


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import pandas as pd

# Carregar modelo de embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  

# Lê o arquivo CSV contendo os dados processados e os armazena em um DataFrame
df = pd.read_csv('dados_processados.csv')  

# Lê o índice FAISS previamente salvo para busca de similaridade
index = faiss.read_index('faiss_index.index') 

# Função para detectar uma categoria ou produto na query do usuário
def detectar_categoria_dinamicamente(query, df):
    query = query.lower()  # Converte a query para minúsculas para comparação
    # Itera sobre as colunas relevantes do DataFrame para verificar se algum valor existe na query
    for coluna in ['product_name', 'product_brand', 'site_category_lv1', 'site_category_lv2']:
        valores_unicos = df[coluna].dropna().unique()  # Remove valores nulos e extrai valores únicos da coluna
        # Itera sobre os valores únicos da coluna para verificar se a query contém um desses valores
        for valor in valores_unicos:
            if valor.lower() in query:  # Verifica se o valor aparece na query
                return valor  # Se encontrado, retorna o valor
    
    return None  # Se nada for encontrado, retorna None

# Função para buscar comentários com base na query do usuário
def buscar_comentarios_dinamicamente(query, df):
    categoria = detectar_categoria_dinamicamente(query, df)  # Chama a função para detectar a categoria ou produto
    
    # Se uma categoria ou produto for detectado
    if categoria:
        print(f"Categoria ou produto detectado: {categoria}")  # Exibe a categoria ou produto detectado
        # Filtra o DataFrame com base na categoria detectada, procurando em várias colunas
        df_filtrado = df[(df['product_name'].str.contains(categoria, case=False, na=False)) |
                         (df['product_brand'].str.contains(categoria, case=False, na=False)) |
                         (df['site_category_lv1'].str.contains(categoria, case=False, na=False)) |
                         (df['site_category_lv2'].str.contains(categoria, case=False, na=False))].copy()  # Usa copy() para evitar problemas com pandas
    
    # Se nenhuma categoria for detectada, usa todos os dados disponíveis
    else:
        print("Nenhuma categoria ou produto detectado, usando todos os produtos.")  
        df_filtrado = df.copy()  # Cria uma cópia do DataFrame original para evitar modificá-lo
    
    return df_filtrado  # Retorna o DataFrame filtrado

# Função para buscar a resposta automaticamente com base na query e nos dados
def buscar_resposta_automatica(query, df, model, index):
    df_filtrado = buscar_comentarios_dinamicamente(query, df)  # Filtra os comentários com base na query
    
    # Se o DataFrame filtrado estiver vazio, retorna uma mensagem indicando que não foram encontrados comentários
    if df_filtrado.empty:
        return "Nenhum comentário encontrado para essa categoria ou produto."
    
    # Garantir que todos os comentários sejam strings
    df_filtrado.loc[:, 'review_text'] = df_filtrado['review_text'].astype(str)  # Converte a coluna 'review_text' para strings
    
    comentarios_filtrados = df_filtrado['review_text'].tolist()  # Converte os comentários filtrados em uma lista
    
    # Verificar se a lista de comentários está vazia
    if not comentarios_filtrados:
        return "Nenhum comentário disponível para busca."  
    
    # Gera embeddings (vetores) dos comentários filtrados
    embeddings_filtrados = model.encode(comentarios_filtrados)
    
    # Define a dimensão dos embeddings para criar o índice FAISS
    dimension = embeddings_filtrados.shape[1]
    index = faiss.IndexFlatL2(dimension)  # Cria um índice FAISS baseado em L2 (distância euclidiana)
    index.add(np.array(embeddings_filtrados, dtype=np.float32))  # Adiciona os embeddings filtrados ao índice FAISS

    # Escreve o índice FAISS gerado em um arquivo para uso posterior
    faiss.write_index(index, 'faiss_index_produto_filtrado.index')

    # Gera o embedding da query do usuário
    query_embedding = model.encode([query])
    # Busca no índice FAISS os k=x comentários mais próximos da query
    D, I = index.search(np.array(query_embedding, dtype=np.float32), k=2)

    # Retorna os x comentários mais próximos com base na similaridade
    resultados = [df_filtrado.iloc[i]['review_text'] for i in I[0]]
    return resultados

query = "celular ?"
resultados = buscar_resposta_automatica(query, df, model, index)

# Exibe os resultados encontrados
for resultado in resultados:
    print(resultado)
